# Final Excercise

In this notebook, you will find the last exercise of the lecture:
You are handed an initial dataset with several features and a univariate target. Next, you have to decide how to proceed. Since you do not have enough data to yet construct a classifier/predictor of any sensible evaluation metrics, the first task is, therefore, to acquire more data. For this purpose you can obtain batches of data according to your own design of experiments, so you will need to decide which experiments you consider necessary to perform. 

You will have four opportunities to acquire more data. Each time you have to decide which experiments to run and send those to Franz Götz-Hahn as a CSV file. The deadlines are 16.06.2023, 23.06.2023, 30.06.2023, and 07.07.2023 and 12:00 (noon). The format in all cases is a table with one row for each choosable feature, and the column entries corresponding to the desired values. Each individual sample will take approximately 30min, so pick a reasonable amount of experiments. For example, you will get the result for 100 experiments roughly 50 hours after the respective deadline. Should the experiment not be conductible, you will get a ``None`` as a result, e.g., if a feature value is out of range.

Once you have your data, you should compare the performance of different classifiers in predicting the targets. The classifiers to compare are [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html), [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC), and [MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html). You should utilize all the different parts of the E2ML lecture that you considern appropriate. This could include Data Preprocessing, Design of Experiments for the batches, deciding on Performance Measures, Statistical Significance Testing of a hypothesis, Design of Experiments for Hyperparameter Optimization.

Should you wish to present the results from this excercise in the oral examination, you need to hand in your entire package until 14.07.2023-23:59 as a GitHub Repository. Send the link to the (public) repository to Franz Götz-Hahn via [E-Mail](mailto:franz.goetz-hahn@uni-kassel.de). Please use the README of the repository to describe the structure of the package, include any required packages in the setup.py, add the data in the data subfolder, save any results in the results subfolder, and include a _descriptive_ jupyter notebook in the notebooks subfolder.

Do note, that the point of this excercise is **not** to achieve the best performance of your models, but rather to document your process and give the motivation behind your chosen approaches, _even the ones that failed_.

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import log_loss
from e2ml.experimentation import perform_bayesian_optimization
from e2ml.preprocessing import PrincipalComponentAnalysis
from sklearn.decomposition import PCA
from e2ml.experimentation import acquisition_ei
from e2ml.models import GaussianProcessRegression
#from e2ml import utils

### **Mollusc Classification** <a class="anchor" id="heart"></a>

Your dataset describes some physical measurements of a specific type of molluscs. Your goal is to predict the `Stage of Life` of the mollusc. The data you can get looks as follows:


| Sex	|Length	|Width	|Height|	Weight	|Non_Shell Weight	|Intestine Weight	|Shell Weight	|Stage of Life |
| ---                           | ----   | ----    | ----    | ----   |----             |----    |---- | ---------- |
| {Male (M), Female (F), Indeterminate (I)} | float (inches)     | float (inches)     |  float (inches)     | float (gram)      | float (gram)              | float (gram)     |  float (gram)     | {Child, Adolescent (Adole), Adult}      |

The table headings are identical to the column names in the corresponding CSV-files. 

We can send out divers that look for molluscs that fit your needs, which will subsequently be analyzed in a laboratory. You can request molluscs with all features except the Stage of Life attribute, as it is the target. The first day of diving has already been completed. After 8 hours of diving, they brought up the following molluscs:

In [3]:
initial_molluscs_data = pd.read_csv('../data/initial_molluscs_data.csv')
initial_molluscs_data

,Sex,Length,Width,Height,Weight,Non_Shell Weight,Intestine Weight,Shell Weight,Stage of Life
0,F,0.450,0.345,0.120,0.4165,0.1655,0.0950,0.1350,Adult
1,F,0.475,0.380,0.145,0.5700,0.1670,0.1180,0.1870,Adole
2,M,0.610,0.485,0.170,1.0225,0.4190,0.2405,0.3600,Adult
3,I,0.430,0.340,0.105,0.4405,0.2385,0.0745,0.1075,Adole
4,M,0.205,0.155,0.045,0.0425,0.0170,0.0055,0.0155,Adult
5,M,0.600,0.475,0.175,1.3445,0.5490,0.2875,0.3600,Child
6,I,0.515,0.390,0.110,0.5310,0.2415,0.0980,0.1615,Adult
7,F,0.625,0.495,0.160,1.1115,0.4495,0.2825,0.3450,Child
8,F,0.650,0.520,0.195,1.6275,0.6890,0.3905,0.4320,Adult
9,F,0.620,0.480,0.165,1.0430,0.4835,0.2210,0.3100,Adult


In [4]:
adults = initial_molluscs_data.loc[initial_molluscs_data["Stage of Life"] == "Adult"]
#print(adults)
adoles = initial_molluscs_data.loc[initial_molluscs_data["Stage of Life"] == "Adole"]
#print(adoles)
children = initial_molluscs_data.loc[initial_molluscs_data["Stage of Life"] == "Child"]
#print(children)
#will 282 haben (5 Tage. 21 Stunden)
full_length = initial_molluscs_data["Length"]
full_width = initial_molluscs_data["Width"]
full_height = initial_molluscs_data["Height"]
full_weight = initial_molluscs_data["Weight"]
full_non_shell_weight = initial_molluscs_data["Non_Shell Weight"]
full_intestine_weight = initial_molluscs_data["Intestine Weight"]
full_shell_weight = initial_molluscs_data["Shell Weight"]
full_sexes = initial_molluscs_data["Sex"]
full_stages = initial_molluscs_data["Stage of Life"]

volume = full_length * full_width * full_height
#print(volume)
weight_volume_quotients = full_weight / volume 
non_shell_quotient = full_non_shell_weight / full_weight
intestine_quotient = full_intestine_weight / full_weight
shell_quotient = full_shell_weight / full_weight

def getOneHotEncoding(data):
    values = np.sort(np.unique(data))
    enc = np.zeros((len(data), len(values)))
    for i, x in enumerate(data):
        enc[i, np.where(values == x)[0][0]] = 1
    return enc

x = np.concatenate((getOneHotEncoding(full_sexes), initial_molluscs_data.values[:,1:-1]), axis=1)
y = getOneHotEncoding(full_stages)

def getNewSamples(old_data:pd.Series, size:int):
    return np.random.normal(old_data.mean(), scale=old_data.std(), size=size)


def score_cross_entropy_loss(mdl, x, y):
    if(len(y.shape) == 1):
        y = getOneHotEncoding(y)
    y_pred = softmax(mdl.predict_proba(x))
    #return cross_entropy_loss(y, y_pred)
    return [log_loss(y[i], y_pred[i])*-1 for i in range(len(y_pred))]

def cross_entropy_loss(y_true, y_pred):
    if(len(y_true.shape) == 1):
        return np.array([np.log(y_pred[i]) for i in y_true])
    elif(len(y_true.shape) == 2):
        return np.array([-(y_true[i] * np.log(y_pred[i])).sum() for i in range(len(y_pred))])
    
def softmax(x):
    x = np.array(x)
    return np.exp(x) / np.exp(x).sum(axis=1).reshape(x.shape[0],-1)

print("mlp")
mlp = MLPClassifier(max_iter=1000)
mlp.fit(x[:10], y[:10])
y_pred = mlp.predict_proba(x[11:])
y_pred = softmax(y_pred)
mlp = MLPClassifier(max_iter=1000)
#cvs_mlp = cross_val_score(mlp, x, y, cv=3, scoring=score_cross_entropy_loss)
#print(cvs_mlp)

y_rfc = full_stages.replace("Adult",0).replace("Adole",1).replace("Child",2)

print("svc")
svc = SVC(kernel="rbf", probability=True)
svc.fit(x[:10], y_rfc[:10])
y_pred = softmax(svc.predict_proba(x[11:]))
svc = SVC(kernel="rbf", probability=True)
#cvs_svc = cross_val_score(svc, x, y_rfc, cv=3, scoring=score_cross_entropy_loss)
#print(cvs_svc)

print("rfc")

rfc = RandomForestClassifier()
rfc.fit(x[:10], y_rfc[:10])
y_pred = softmax(rfc.predict_proba(x[11:]))
rfc = RandomForestClassifier()
#cvs_rfc = cross_val_score(rfc, x, y_rfc, cv=3, scoring=score_cross_entropy_loss)
#print(f"{cvs_rfc=}")

def objectiveFunction(x, y):
    rfc = RandomForestClassifier()
    cvs_rfc = cross_val_score(rfc, x, y_rfc, cv=3, scoring=score_cross_entropy_loss)
    svc = SVC(kernel="rbf", probability=True)
    cvs_svc = cross_val_score(svc, x, y_rfc, cv=3, scoring=score_cross_entropy_loss)
    mlp = MLPClassifier(max_iter=1000)
    cvs_mlp = cross_val_score(mlp, x, y, cv=3, scoring=score_cross_entropy_loss)
    return (cvs_rfc.mean() + cvs_svc.mean() + cvs_mlp.mean()) / 3




mlp
svc
rfc


/Users/ben/anaconda3/envs/e2ml-env/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [5]:
def getDict(size:int):
    d = {}
    d["Sex"] = np.random.choice(full_sexes, size)
    d["Length"] = getNewSamples(full_length, size)
    d["Width"] = getNewSamples(full_width, size)
    d["Height"] =  getNewSamples(full_height, size)
    d["Weight"] = d["Height"] * d["Width"] * d["Length"] * np.random.normal(weight_volume_quotients.mean(), weight_volume_quotients.std())
    d["Non_Shell Weight"] = d["Weight"] * np.random.normal(non_shell_quotient.mean(), non_shell_quotient.std())
    d["Intestine Weight"] = d["Weight"] * np.random.normal(intestine_quotient.mean(), intestine_quotient.std())
    d["Shell Weight"] = d["Weight"] * np.random.normal(shell_quotient.mean(), shell_quotient.std())
    return d
d = getDict(282)
print(f"{d['Weight'].mean()=} {d['Weight'].std()=}")
print(f"{full_weight.mean()=} {full_weight.std()=}")
print(f"{d['Non_Shell Weight'].mean()=} {d['Non_Shell Weight'].std()=}")
print(f"{full_non_shell_weight.mean()=} {full_non_shell_weight.std()=}")
print(f"{d['Intestine Weight'].mean()=} {d['Intestine Weight'].std()=}")
print(f"{full_intestine_weight.mean()=} {full_intestine_weight.std()=}")
print(f"{d['Shell Weight'].mean()=} {d['Shell Weight'].std()=}")
print(f"{full_shell_weight.mean()=} {full_shell_weight.std()=}")
orignial_diffs = full_weight - full_intestine_weight - full_non_shell_weight - full_shell_weight
new_diffs = d["Weight"] - d["Intestine Weight"] - d["Non_Shell Weight"] - d["Shell Weight"]
print(orignial_diffs)
print(new_diffs)
print(f"{orignial_diffs.mean()=} {orignial_diffs.std()=}")
print(f"{new_diffs.mean()=} {new_diffs.std()=}")
print(pd.DataFrame(d))

d['Weight'].mean()=0.8814305281809354 d['Weight'].std()=0.3969577521064493
full_weight.mean()=0.877875 full_weight.std()=0.4965779730649894
d['Non_Shell Weight'].mean()=0.34999677479986413 d['Non_Shell Weight'].std()=0.1576232369166837
full_non_shell_weight.mean()=0.37634375 full_non_shell_weight.std()=0.22438981845217487
d['Intestine Weight'].mean()=0.19395211472815496 d['Intestine Weight'].std()=0.08734754812460532
full_intestine_weight.mean()=0.18753124999999998 full_intestine_weight.std()=0.10964373652121372
d['Shell Weight'].mean()=0.325399163721667 d['Shell Weight'].std()=0.14654554889861507
full_shell_weight.mean()=0.25493750000000004 full_shell_weight.std()=0.13599557284460892
0     0.0210
1     0.0980
2     0.0030
3     0.0200
4     0.0045
5     0.1480
6     0.0300
7     0.0345
8     0.1160
9     0.0285
10    0.0545
11    0.0180
12    0.1795
13    0.0575
14    0.0610
15    0.0710
dtype: float64
[0.00116328 0.00267848 0.0115252  0.01723005 0.01488373 0.0046356
 0.01578373 0.011

In [6]:
print(pd.read_csv("FirstBatchBen.csv"))

     Unnamed: 0 Sex    Length     Width    Height    Weight  Non_Shell Weight  \
0             0   M  0.471305  0.290373  0.106767  0.303367          0.123269   
1             1   F  0.731545  0.478463  0.170384  1.238197          0.503123   
2             2   I  0.615436  0.235762  0.065890  0.198494          0.080655   
3             3   F  0.451397  0.144832  0.161660  0.219429          0.089162   
4             4   F  0.689066  0.398501  0.175774  1.002112          0.407194   
..          ...  ..       ...       ...       ...       ...               ...   
277         277   F  0.555961  0.375405  0.131120  0.568176          0.230870   
278         278   F  0.605268  0.583672  0.065907  0.483415          0.196429   
279         279   M  0.529225  0.241043  0.140752  0.372786          0.151476   
280         280   F  0.411005  0.499416  0.121998  0.519914          0.211260   
281         281   M  0.603832  0.480075  0.120224  0.723582          0.294017   

     Intestine Weight  Shel

In [20]:
full_replaced = initial_molluscs_data.replace("F",0).replace("I",1).replace("M",2)
x_full_replaced = full_replaced.values[:,:-1]
pca = PCA(2)
pca = pca.fit(x_full_replaced)
print(pca.explained_variance_ratio_)
print(pca.singular_values_)

x_pca = pca.transform(x_full_replaced)
print(x_pca)

#new valuerange is -1.5, 1.5

print("mlp")
mlp = MLPClassifier(max_iter=1000)
mlp.fit(x_pca[:10], y[:10])
y_pred = mlp.predict_proba(x_pca[11:])
y_pred = softmax(y_pred)
mlp = MLPClassifier(max_iter=1000)
#cvs_mlp = cross_val_score(mlp, x_pca, y, cv=3, scoring=score_cross_entropy_loss)
#print(cvs_mlp)

y_rfc = full_stages.replace("Adult",0).replace("Adole",1).replace("Child",2)

print("svc")
svc = SVC(kernel="rbf", probability=True)
svc.fit(x_pca[:10], y_rfc[:10])
y_pred = softmax(svc.predict_proba(x_pca[11:]))
svc = SVC(kernel="rbf", probability=True)
#cvs_svc = cross_val_score(svc, x_pca, y_rfc, cv=3, scoring=score_cross_entropy_loss)
#print(cvs_svc)

print("rfc")

rfc = RandomForestClassifier()
rfc.fit(x_pca[:10], y_rfc[:10])
y_pred = softmax(rfc.predict_proba(x_pca[11:]))
rfc = RandomForestClassifier()
#cvs_rfc = cross_val_score(rfc, x_pca, y_rfc, cv=3, scoring=score_cross_entropy_loss)
#print(f"{cvs_rfc=}")

def objectiveFunctionRFC(x, y_rfc):
    rfc = RandomForestClassifier()
    rfc.fit(x,y_rfc)
    return score_cross_entropy_loss(rfc, x, y_rfc)

def objectiveFunctionSVC(x,y_rfc):
    svc = SVC(kernel="rbf", probability=True)
    svc.fit(x, y_rfc)
    return score_cross_entropy_loss(svc, x, y_rfc)

def objectiveFunctionMLP(x, y):
    mlp = MLPClassifier(max_iter=1000)
    mlp.fit(x,y)
    return score_cross_entropy_loss(mlp, x, y)


x_acquired = x_pca
y_acquired_rfc = objectiveFunctionRFC(x_acquired, y_rfc)
print(y_acquired_rfc)
y_acquired_svc = objectiveFunctionSVC(x_pca, y_rfc)
print(y_acquired_svc)
y_acquired_mlp = objectiveFunctionMLP(x_pca, y)
print(y_acquired_mlp)

metrics_dict = {'gamma': 50, 'metric': 'rbf'}
gpr = GaussianProcessRegression(metrics_dict=metrics_dict)
gpr.fit(x_acquired, y_acquired_rfc)

x1_new = np.linspace(-1.5, 1.5, 100)
x_mesh,y_mesh = np.meshgrid(x1_new, x1_new)

x_cand = np.stack((x_mesh, y_mesh), axis=2).reshape(-1,2)
print(x_cand.shape)
means, stds = gpr.predict(x_cand, True)
scores = acquisition_ei(means, stds, max(y_acquired_rfc))
print(scores)
print(scores.mean())
print(scores.std())
print(scores.min())
print(scores.max())

nextidx = np.argsort(scores)

print(scores[nextidx[-10:]])
print(x_cand[nextidx[-10:]])
print(pca.inverse_transform(x_cand[nextidx[-10:]]))

[0.70537116 0.28817022]
[3.54073097 2.26312547]
[[-0.74840581 -0.62740424]
 [-0.76548386 -0.47050949]
 [ 1.15757455  0.32628467]
 [ 0.24195993 -0.4856955 ]
 [ 1.29797218 -0.90029362]
 [ 1.12356131  0.64986098]
 [ 0.22758857 -0.36834205]
 [-0.84014791  0.2007329 ]
 [-0.90098976  0.77220098]
 [-0.83319121  0.13402528]
 [-0.79940449 -0.16651914]
 [ 0.2713473  -0.7486818 ]
 [-0.92209266  0.9578332 ]
 [-0.78875156 -0.27996054]
 [ 1.15739731  0.34398507]
 [ 1.12106611  0.66248328]]
mlp


/Users/ben/anaconda3/envs/e2ml-env/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


svc
rfc
[-0.4358830738970912, -0.40557995111363737, -0.3809508318793582, -0.4678996098629882, -0.35372802568051775, -0.3849703293823343, -0.4678996098629882, -0.5027245280704169, -0.4993771020092204, -0.46061647923754706, -0.39723330483646063, -0.37696525203047004, -0.5027245280704169, -0.389023843667635, -0.3809508318793582, -0.3849703293823343]
[-0.5286510821909896, -0.6900511382590343, -0.5930193290531345, -0.6020448600382837, -0.6127667588754827, -0.6701073104564763, -0.603862428064263, -0.7110183150912901, -0.5690158753733245, -0.5401945972422366, -0.6883861223109595, -0.6092462290550656, -0.6346811264131905, -0.6907097237501878, -0.5912339367432035, -0.6711891418270112]


/Users/ben/anaconda3/envs/e2ml-env/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


[-0.5265047663984878, -0.47844504061670684, -0.35940399472868, -0.607526623151755, -0.342794570990995, -0.36351213287206496, -0.5473121257181087, -0.5823432204546927, -0.507411321940879, -0.585393980635796, -0.449050567917455, -0.37537023391204744, -0.489096065996955, -0.394520680784707, -0.36531508518459876, -0.35979553972814227]
(10000, 2)
[0.60069514 0.60069514 0.60069514 ... 0.60069514 0.60069514 0.60069514]
0.5705163421542732
0.08854439447994962
8.472837465397392e-05
0.6006951355493214
[0.60069514 0.60069514 0.60069514 0.60069514 0.60069514 0.60069514
 0.60069514 0.60069514 0.60069514 0.60069514]
[[0.16666667 1.04545455]
 [0.31818182 1.01515152]
 [0.25757576 1.01515152]
 [0.22727273 1.01515152]
 [0.1969697  1.01515152]
 [0.16666667 1.01515152]
 [0.13636364 1.01515152]
 [0.10606061 1.01515152]
 [0.28787879 1.01515152]
 [1.5        1.5       ]]
[[1.09279335 0.71898485 0.58725877 0.20905978 1.7393944  0.75870448
  0.37209006 0.48743274]
 [1.2400081  0.7070962  0.57773746 0.20536868 1